In [1]:
import os,sys,string, time
import ROOT
from math import *
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from root_pandas import read_root
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import pickle

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')

Welcome to JupyROOT 6.24/06
Success


# Loading in the "results" dataframe after full selection

In [2]:
Run = "run3" #so far either "run1" or "run3"

HNL_masses = Constants.HNL_mass_samples

loc_pkl = f'pkl_files/{Run}/current_files/Results/'

overlay_results = pd.read_pickle(loc_pkl+"overlay_results_NEW.pkl") #This will contain all of the BDT output scores and rse_id

loc_hists = 'bdt_output/'

filename = f'logit_FINAL_1.root' #CHANGE THIS IF CHANGING FILE INPUT

bins_dict = {}
for HNL_mass in HNL_masses:
    hist_placeholder = uproot.open(loc_hists+f'{Run}_{HNL_mass}MeV_'+filename)
    bins_dict[HNL_mass] = hist_placeholder['bkg_overlay'].to_numpy()[1] #A tuple of bin edges

# Reading in the overlay .root file with reweight branches

In [3]:
sys_variables = Variables.sys_vars + Variables.event_vars + Variables.weight_related

if Run == "run1":
    NuMI_MC_overlay = uproot3.open('../NuMI_MC/SLIMMED_neutrinoselection_filt_run1_overlay.root')['nuselection/NeutrinoSelectionFilter']
    Norm = Constants.SF_overlay_run1
elif Run == "run3":
    NuMI_MC_overlay = uproot3.open('../NuMI_MC/SLIMMED_neutrinoselection_filt_run3_overlay.root')['nuselection/NeutrinoSelectionFilter']
    Norm = Constants.SF_overlay_run3

df_overlay_weights = NuMI_MC_overlay.pandas.df(sys_variables, flatten=False) #Perhaps I can do this in a more clever way than just making a dataframe


# Keeping only the events which pass selection in the weight dataframe

In [4]:
def make_unique_ev_id(df): #df must have 'run', 'sub' and 'evt' branches
    if pd.Series(['run', 'sub', 'evt']).isin(df.columns).all():
        rse_list = []
        for entry in df.index: #Looping over all events in the dataframe
            rse = str(df['run'][entry]) + "_" + str(df['sub'][entry]) + "_" + str(df['evt'][entry])
            rse_list.append(rse)
        df['rse_id'] = rse_list #Writing a new branch with the unique event id
        return df.copy()
    else:
        print("Dataframe needs \"run\", \"sub\" and \"evt\" columns.")
        return 0
    
def check_duplicate_events(df):
    rse_list = df['rse_id'].to_list()

    seen = set()
    dupes = []

    for x in rse_list:
        if x in seen:
            dupes.append(x)
        else:
            seen.add(x)
    print("Number of duplicates is " + str(len(dupes)))
    print("Number of unique events is " + str(len(seen)))

overlay_results_rse = make_unique_ev_id(overlay_results)
df_overlay_weights_rse = make_unique_ev_id(df_overlay_weights)

#Deleting any duplicates of events, should be able to avoid if correctly filtered for one event per row
overlay_results_rse.drop_duplicates(subset=['rse_id'], keep='first', inplace=True)

print("Number of events in weights file is " + str(len(df_overlay_weights_rse)))
print("Number of events in results file is " + str(len(overlay_results_rse)))

Number of events in weights file is 748702
Number of events in results file is 8849


In [5]:
#Keeping only those events in the final selection
filtered_weights = df_overlay_weights_rse.loc[(df_overlay_weights_rse['rse_id'].isin(overlay_results_rse['rse_id']))]

print("Number of events in the filtered weights file is " + str(len(filtered_weights)))
print("Number of events in results file is " + str(len(overlay_results_rse)))

Number of events in the filtered weights file is 8849
Number of events in results file is 8849


## Calculating uncertainty for a BDT input variable

In [6]:
print(filtered_weights.keys())
print(overlay_results_rse.keys())

Index(['weightsPPFX', 'weightsGenie', 'weightsReint', 'run', 'sub', 'evt',
       'weightSplineTimesTune', 'ppfx_cv', 'npi0', 'rse_id'],
      dtype='object')
Index(['run', 'sub', 'evt', 'weightSplineTimesTune', 'ppfx_cv', 'npi0',
       'weight', 'BDT_output_2MeV', 'BDT_output_10MeV', 'BDT_output_20MeV',
       'BDT_output_50MeV', 'BDT_output_100MeV', 'BDT_output_150MeV',
       'BDT_output_180MeV', 'BDT_output_200MeV', 'BDT_output_220MeV',
       'BDT_output_240MeV', 'BDT_output_245MeV', 'rse_id'],
      dtype='object')


In [7]:
BDT_score_list = []

# HNL_masses = [20, 50, 100, 150, 180, 200] #Should get rid of this once made overlay branches with all results

for HNL_mass in HNL_masses:
    BDT_score_list.append(f'BDT_output_{HNL_mass}MeV')

just_score_df = overlay_results_rse[BDT_score_list + ['rse_id','weight']].copy()

final_merged = pd.merge(filtered_weights,just_score_df, how='outer', on=['rse_id'])

## Merging into one dataframe with results and weights

In [8]:
BDT_score_list = []

# HNL_masses = [20, 50, 100, 150, 180, 200] #Should get rid of this once made overlay branches with all results

for HNL_mass in HNL_masses:
    BDT_score_list.append(f'BDT_output_{HNL_mass}MeV')

just_score_df = overlay_results_rse[BDT_score_list + ['rse_id','weight']].copy()

final_merged = pd.merge(filtered_weights,just_score_df, how='outer', on=['rse_id']) #This will have the reweighting branches AND BDT score branches


# Plotting the BDT score with all different multisims

In [9]:
def All_reweight_err(df, var_name, BINS, x_range, Norm, logit=True):
    results_dict = {}
    n_bins = len(BINS)-1
    for Multisim in Constants.Multisim_univs:
        Nuniverse = Constants.Multisim_univs[Multisim]
        n_tot = np.empty([Nuniverse, n_bins])
        n_cv_tot = np.empty(n_bins)
        n_tot.fill(0)
        n_cv_tot.fill(0)
        
        if logit == True:
            variable = Functions.logit(df[var_name]) #The BDT output score
        else:
            variable = df[var_name] #The BDT output score
        syst_weights = df[Multisim] #An array of length of the number of events, each entry is an array of length Nunivs
        spline_fix_cv  = df["weight"]*Norm
        spline_fix_var = df["weight"]*Norm
        
        s = syst_weights
        df_weights = pd.DataFrame(s.values.tolist())
        n_cv, bins = np.histogram(variable, range=x_range, bins=BINS, weights=spline_fix_cv)
        n_cv_tot += n_cv
        
        if(Multisim == "weightsGenie"): #special treatment as ["weightSplineTimesTune"] is included in genie weights
            if not df_weights.empty:
                for i in range(Nuniverse):
                    weight = df_weights[i].values / 1000.
                    weight[weight == 1]= df["weightSplineTimesTune"].iloc[weight == 1]
                    weight[np.isnan(weight)] = df["weightSplineTimesTune"].iloc[np.isnan(weight)]
                    weight[weight > 50] = df["weightSplineTimesTune"].iloc[weight > 50] # why 30 not 50?
                    weight[weight <= 0] = df["weightSplineTimesTune"].iloc[weight <= 0]
                    weight[weight == np.inf] = df["weightSplineTimesTune"].iloc[weight == np.inf]
                
                    n, bins = np.histogram(variable, 
                                           weights=np.nan_to_num(weight*spline_fix_var/df["weightSplineTimesTune"]), range=x_range, bins=BINS)
                    n_tot[i] += n
                    
        if(Multisim == "weightsPPFX"): #special treatment as ["PPFXPcv"] is included in ppfx weights
            if not df_weights.empty:
                for i in range(Nuniverse):
                    weight = df_weights[i].values / 1000.
                    weight[weight == 1]= df["ppfx_cv"].iloc[weight == 1]
                    weight[np.isnan(weight)] = df["ppfx_cv"].iloc[np.isnan(weight)]
                    weight[weight > 100] = df["ppfx_cv"].iloc[weight > 100]
                    weight[weight < 0] = df["ppfx_cv"].iloc[weight < 0]
                    weight[weight == np.inf] = df["ppfx_cv"].iloc[weight == np.inf]
                
                    n, bins = np.histogram(variable, weights=weight*np.nan_to_num(spline_fix_var/df["ppfx_cv"]), range=x_range, bins=BINS)
                    n_tot[i] += n
        
        if(Multisim == "weightsReint"):
            if not df_weights.empty:
                for i in range(Nuniverse):
                    weight = df_weights[i].values / 1000.
                    weight[np.isnan(weight)] = 1
                    weight[weight > 100] = 1
                    weight[weight < 0] = 1
                    weight[weight == np.inf] = 1
                    n, bins = np.histogram(variable, weights=weight*spline_fix_var, range=x_range, bins=BINS)
                    n_tot[i] += n
        cov = np.empty([len(n_cv), len(n_cv)])
        cov.fill(0)

        for n in n_tot:
            for i in range(len(n_cv)):
                for j in range(len(n_cv)):
                    cov[i][j] += (n[i] - n_cv_tot[i]) * (n[j] - n_cv_tot[j])

        cov /= Nuniverse
        results_dict[Multisim] = [cov,n_cv_tot,n_tot,bins]
    return results_dict

In [10]:
# HNL_mass = 50
logit = True
results_dict = {}
for HNL_mass in HNL_masses:
    print(f"Calculating {HNL_mass}MeV uncertainties.")
    results_dict[HNL_mass] = All_reweight_err(final_merged, f'BDT_output_{HNL_mass}MeV', bins_dict[HNL_mass],
                                    [bins_dict[HNL_mass][0], bins_dict[HNL_mass][-1]], Norm, logit=logit)

print("Done!")

Calculating 2MeV uncertainties.
Calculating 10MeV uncertainties.
Calculating 20MeV uncertainties.
Calculating 50MeV uncertainties.
Calculating 100MeV uncertainties.
Calculating 150MeV uncertainties.
Calculating 180MeV uncertainties.
Calculating 200MeV uncertainties.
Calculating 220MeV uncertainties.
Calculating 240MeV uncertainties.
Calculating 245MeV uncertainties.
Done!


In [11]:
def Plot_multisim(results_dict, HNL_mass, savefig=False, display=False):
    for Multisim in results_dict:
        Nuniverse = Constants.Multisim_univs[Multisim]
        cov = results_dict[Multisim][0]
        cv = results_dict[Multisim][1]
        n_tot = results_dict[Multisim][2]
        bins = results_dict[Multisim][3]
        xlims = [min(bins), max(bins)]
        
        fig,ax = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [3, 1]}, figsize=[10,10],dpi=200)
        plt.sca(ax[0])

        # bins=np.linspace(0,1.0,21)

        bins_cent=(bins[:-1]+bins[1:])/2
        bins_centlong=np.tile(bins_cent,Nuniverse)

        nybins=70

        plt.title(Multisim + " Variations",fontsize=20)

        plt.hist(bins_cent,bins,weights=cv,color="red",histtype="step",label="Central Value",lw=2,linestyle='-')
        plt.legend()
        bins_cent=(bins[:-1]+bins[1:])/2
        bins_centlong=np.tile(bins_cent,Nuniverse)

        plt.ylabel("Events")
        plt.hist2d(bins_centlong,n_tot.flatten(),bins=[bins,nybins],cmin=1,range=[xlims,[0,max(cv)*1.4]],rasterized=True)

        plt.colorbar(pad=0,use_gridspec=True)
        #fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),pad=0,use_gridspec=True)
        plt.ylim(0,max(cv)*1.4)
        # plt.yscale("log")

        plt.sca(ax[1])
        
        #ax[1].tick_params(labelright=False, length=0)
        pos = ax[0].get_position()
        pos2 = ax[1].get_position()
        ax[1].set_position([pos.x0,pos2.y0,pos.width,pos2.height])
        
        plt.hist(bins_cent,bins,weights=np.sqrt(np.diag(cov))/cv*100,color="black",histtype="step",lw=3,linestyle='-')
        maxy = 1.5*max(plt.hist(bins_cent,bins,weights=np.sqrt(np.diag(cov))/cv*100,color="black",histtype="step",lw=3,linestyle='-')[0])
        plt.ylim(0,maxy)
        plt.ylabel("% Uncertainity")
        #plt.yticks([])
        plt.xlabel(f"BDT score ({HNL_mass} MeV HNL) ",fontsize=25)
        # plt.tight_layout()
        if savefig == True:
            plt.savefig("plots/Sys_uncertainty/Overlay/Multisim/bkg_multisim_" + Run + "_" + str(HNL_mass) + "_MeV_" + Multisim + ".pdf")
            plt.savefig("plots/Sys_uncertainty/Overlay/Multisim/bkg_multisim_" + Run + "_" + str(HNL_mass) + "_MeV_" + Multisim + ".png")
        if display == False:
            plt.close()

In [12]:
HNL_masses = Constants.HNL_mass_samples
for HNL_mass in HNL_masses:
    Plot_multisim(results_dict[HNL_mass], HNL_mass, savefig=False, display=False)

/home/david/jupyter/envs/ana/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
/home/david/jupyter/envs/ana/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/home/david/jupyter/envs/ana/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
/home/david/jupyter/envs/ana/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/home/david/jupyter/envs/ana/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
/home/david/jupyter/envs/ana/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/home/david/jupyter/envs/ana/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
/home/david/jupyter/

## Calculating and saving total uncertainty for each reweighting

In [13]:
ppfx_unc_dict, ppfx_unc_frac_dict = {}, {}
genie_unc_dict, genie_unc_frac_dict = {}, {}
reint_unc_dict, reint_unc_frac_dict = {}, {}

for HNL_mass in HNL_masses:
    
    diagonal_ppfx = np.diag(results_dict[HNL_mass]["weightsPPFX"][0]).copy()
    diagonal_genie = np.diag(results_dict[HNL_mass]["weightsGenie"][0]).copy()
    diagonal_reint = np.diag(results_dict[HNL_mass]["weightsReint"][0]).copy()
    
    num_ppfx = results_dict[HNL_mass]["weightsPPFX"][1].copy()
    num_genie = results_dict[HNL_mass]["weightsPPFX"][1].copy()
    num_reint = results_dict[HNL_mass]["weightsReint"][1].copy()
    if np.any(diagonal_ppfx==0):
        print(f"{HNL_mass}MeV contains zeros, setting error to that of prev bin")
    index = np.where(diagonal_ppfx == 0)
    diagonal_ppfx[diagonal_ppfx==0] = 1
    diagonal_genie[diagonal_genie==0] = 1
    diagonal_reint[diagonal_reint==0] = 1
    
    num_ppfx[num_ppfx==0] = 1
    num_genie[num_genie==0] = 1
    num_reint[num_reint==0] = 1
    
    ppfx_unc_dict[HNL_mass] = np.sqrt(diagonal_ppfx)
    ppfx_unc_frac_dict[HNL_mass] = ppfx_unc_dict[HNL_mass]/num_ppfx

    genie_unc_dict[HNL_mass] = np.sqrt(diagonal_genie)
    genie_unc_frac_dict[HNL_mass] = genie_unc_dict[HNL_mass]/num_genie

    reint_unc_dict[HNL_mass] = np.sqrt(diagonal_reint)
    reint_unc_frac_dict[HNL_mass] = reint_unc_dict[HNL_mass]/num_reint
    
    for i in index:
        ppfx_unc_frac_dict[HNL_mass][i] = ppfx_unc_frac_dict[HNL_mass][i-1]
        genie_unc_frac_dict[HNL_mass][i] = genie_unc_frac_dict[HNL_mass][i-1]
        reint_unc_frac_dict[HNL_mass][i] = reint_unc_frac_dict[HNL_mass][i-1]
        
        ppfx_unc_dict[HNL_mass][i] = ppfx_unc_dict[HNL_mass][i-1]
        genie_unc_dict[HNL_mass][i] = genie_unc_dict[HNL_mass][i-1]
        reint_unc_dict[HNL_mass][i] = reint_unc_dict[HNL_mass][i-1]


2MeV contains zeros, setting error to that of prev bin
10MeV contains zeros, setting error to that of prev bin
20MeV contains zeros, setting error to that of prev bin


## Saving reweighting uncertainties as .root files

In [14]:
loc_hists = "Uncertainties/"

for HNL_mass in HNL_masses:
    bins_cent=(bins_dict[HNL_mass][:-1]+bins_dict[HNL_mass][1:])/2

    values_dict = {'ppfx_uncertainty': ppfx_unc_dict[HNL_mass], 'Genie_uncertainty':genie_unc_dict[HNL_mass],
                   'Reinteraction_uncertainty':reint_unc_dict[HNL_mass],
                   'ppfx_uncertainty_frac': ppfx_unc_frac_dict[HNL_mass], 'Genie_uncertainty_frac':genie_unc_frac_dict[HNL_mass],
                   'Reinteraction_uncertainty_frac':reint_unc_frac_dict[HNL_mass]} 
    hist_samples = {}

    #make array with all values 1, then weight by value
    for name in values_dict:
        test_hist = np.histogram(bins_cent, weights=values_dict[name], bins=bins_dict[HNL_mass], 
                                 range=[bins_dict[HNL_mass][0], bins_dict[HNL_mass][-1]])
        hist_samples[name] = test_hist

    stop_writing = False
    dont_save = []
    with uproot.open(loc_hists+f'{Run}_{HNL_mass}MeV_'+filename) as file: #Check what is already in the file (read-only)
        for name in hist_samples:
            if str(name)+";1" in file.keys():
                # print(str(name) + f" ALREADY EXISTS in {HNL_mass}MeV file, not re-saving")
                dont_save.append(name)

    with uproot.update(loc_hists+f'{Run}_{HNL_mass}MeV_'+filename) as file: #Add new hists into the file
        # del file['ppfx_uncertainty']

        for name in hist_samples:
            # if stop_writing == True:
            #     print("Not saving hist")
            #     break
            if name in dont_save:
                # print(f"Not saving {name}")
                continue
            else:
                file[name] = hist_samples[name]
print("Done")

Done


## Can't figure out how to delete hists

In [ ]:
#Trying to delete
# for HNL_mass in HNL_masses:
HNL_mass = 220
file = uproot.update(loc_hists+f'{Run}_{HNL_mass}MeV_'+filename) #Add new hists into the file
        # del file['ppfx_uncertainty']
print(file.keys())
# del file['ppfx_uncertainty_frac']
print(file.keys())

# for name in hist_samples:
#     del file[name]
    
        # for name in hist_samples:
        #     if str(name)+";1" in file.keys():
        #         print("Trying to delete " + name+";1")
        #         del file[name+";1"]
        
            # if stop_writing == True:
            #     print("Not saving hist")
            #     break
            # if name in dont_save:
            #     print(f"Not saving {name}")
            # else:
            # del file[name]
            # file[name] = hist_samples[name]

## End of code

In [ ]:
def Save_one_hist(hist,name,nbins,xlims):
    # tData = ROOT.TH1F(name,name,nbins,xlims[0],xlims[1])
    tData = ROOT.TH1F(name,name,len(nbins)-1,array("d",nbins))
    for i in range(len(nbins)-1):
        tData.SetBinContent(i+1,hist['hist'][i])
        tData.SetBinError(i+1,hist['err'][i])
    return tData
    
def SaveToRoot_new(nbins,xlims,hist_samples,fileName='test.root'): 
    rFile = ROOT.TFile(f'bdt_output/{fileName}','RECREATE')
    for name in hist_samples:
        tData = Save_one_hist(hist_samples[name],name,nbins,xlims)
        rFile.Write()
    #rFile.Write()
    rFile.Close()

def make_stat_err(hist, SF):
    stat_err = []
    for i in range(0,len(hist[0])):
        error = np.sqrt(hist[0][i])*np.sqrt(SF)
        stat_err.append(error)
    return stat_err

In [ ]:
stop_writing = False
with uproot.open(loc_hists+f'{Run}_{HNL_mass}MeV_logit_top_20_merged.root') as file: #Check what is already in the file (read-only)
    print(type(file.keys()))
    for name in hist_samples:
        if str(name)+";1" in file.keys():
            print(str(name) + " ALREADY EXISTS, stopping")
            stop_writing = True
            break
        else:
            print(name)
            
with uproot.update(loc_hists+f'{Run}_{HNL_mass}MeV_logit_top_20_merged.root') as file: #Add new hists into the file
    # del file['ppfx_uncertainty']
        
    for name in hist_samples:
        if stop_writing == True:
            print("Not saving hist")
            break
        file[name] = hist_samples[name]

In [ ]:
# with uproot.update(loc_hists+f'{Run}_{HNL_mass}MeV_logit_top_20_merged.root') as file:
# # file = uproot.update(loc_hists+f'{Run}_{HNL_mass}MeV_logit_top_20_merged.root')
#     print(file.keys())
#     del file['ppfx_uncertainty;1']
#     print(file.keys())

In [ ]:
def SaveToRoot(nbins,xlims,uncertainty,name,fileName='test.root'):
    nBins = nbins
    binLimits = xlims
  ### Save files 
    rFile = ROOT.TFile(f'Uncertainties/{fileName}','RECREATE')
    tData1 = ROOT.TH1F(name,name,nBins,binLimits[0],binLimits[1])
    for i in range(nBins):
        tData1.SetBinContent(i+1,uncertainty[i])
    rFile.Write()
    rFile.Close()
    
def SaveToRoot_new(nbins,xlims,hist_samples,fileName='test.root'): 
    rFile = ROOT.TFile(f'Uncertainties/{fileName}','RECREATE')
    for name in hist_samples:
        tData1 = ROOT.TH1F(name,name,nbins,xlims[0],xlims[1])
        for i in range(nbins): 
            tData1.SetBinContent(i+1,hist_samples[name][i])
        rFile.Write()
    rFile.Close()
    
def SaveToRoot_INPUT(bins_dict_single,hist_samples,fileName='test.root'): 
    rFile = ROOT.TFile(f'Uncertainties/{fileName}','RECREATE')
    for name in hist_samples:
        tData1 = ROOT.TH1F(name,name,len(bins_dict_single)-1,bins_dict_single[0],bins_dict_single[1])
        for i in range(len(bins_dict_single)-1): 
            tData1.SetBinContent(i+1,hist_samples[name][i])
        rFile.Write()
    rFile.Close()

In [ ]:
full_hists = {} #Full hists here, with all old entries and new uncertainties added

SaveToRoot_INPUT(bins_dict_single,hist_samples,fileName=f'Test_3_{HNL_mass}MeV.root')

In [ ]:
reweight_errs = {'ppfx_uncertainty': ppfx_unc, 'Genie_uncertainty':genie_unc, 'Reinteraction_uncertainty':reint_unc}
reweight_errs_frac = {'ppfx_uncertainty_frac': ppfx_unc_frac, 'Genie_uncertainty_frac':genie_unc_frac, 
                      'Reinteraction_uncertainty_frac':reint_unc_frac}

SaveToRoot(20,[0,1.0],ppfx_unc,'ppfx_uncertainty',fileName=f'Test_1_{HNL_mass}MeV.root')

SaveToRoot_new(20,[0,1.0],reweight_errs,fileName=Run+f'_Test_2_{HNL_mass}MeV.root')
SaveToRoot_new(20,[0,1.0],reweight_errs_frac,fileName=Run+f'_Test_2_frac_{HNL_mass}MeV.root')